In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from skmultilearn.adapt import MLkNN

from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors


from sklearn.metrics import accuracy_score, classification_report

In [2]:
pd.set_option('display.max_columns', None)

# With grapes

In [18]:
def create_binary_df_2(file_path):
    mlb_harm = MultiLabelBinarizer(sparse_output=False)
    mlb_grape = MultiLabelBinarizer(sparse_output=False)

    wine_df = pd.read_csv(file_path)
    
    #Drop addional columns not used for model
    wine_df = wine_df.drop(columns=['WineName', 'WineID','Code','Country','RegionID','RegionName','WineryID','Website','Vintages', 'WineryName'])
    
    # Binary encode grapes
    wine_df_bin = wine_df.join(pd.DataFrame(
        mlb_grape.fit_transform(eval(element) for element in wine_df.Grapes),
        index=wine_df.index,
        columns=mlb_grape.classes_
        ))
    wine_df_bin.drop(columns=['Grapes'], inplace=True)

    # Create a list of the kind of grapes that are mentioned less then 2.000 times
    grapes_list = wine_df_bin.iloc[:,16:].sum() # sum the number of times a grape is mentioned via column
    final_column_grapes = wine_df_bin.shape[1]
    
    # Binary encode Harmonize(kinds of food)
    wine_df_bin = wine_df_bin.join(pd.DataFrame(
        mlb_harm.fit_transform(eval(element) for element in wine_df.Harmonize),
        index=wine_df.index,
        columns=mlb_harm.classes_
        ))
    wine_df_bin.drop(columns=['Harmonize'], inplace=True)
    
    # Create a list of the kind of grapes that are mentioned less then 2.000 times
    harm_list = wine_df_bin.iloc[:,(final_column_grapes+1):].sum() # sum the number of times a food is mentioned via column
    harm_to_drop = harm_list[harm_list<=15_000].index.to_list() # create a list withe kind of food mentioned less then 50 times
    wine_df_bin.drop(columns=harm_to_drop, inplace=True) # drop columns with food not mentioned more then 50 times
    wine_df_bin = wine_df_bin[wine_df_bin.iloc[:,(final_column_grapes+1):].eq(1).any(axis=1)] # drop wines which are not represented by a food anymore

    return wine_df_bin, final_column_grapes

In [19]:
def save_cleaned_csv(df):
    df.to_csv('~/code/ArjanAngenent/VinoDine/raw_data/cleaned_data.csv', index=False)
    return None

In [20]:
def create_X_train_y_train_2(df, grape_column, test_size=0.3):
    X = df.iloc[:,:grape_column-1]
    X = X.drop(columns = 'Elaborate')
    y = df.iloc[:,grape_column:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=test_size)
    return X_train, X_test, y_train, y_test

In [21]:
def train_model_2(X_train, y_train):
    # Create binary classifier
    # Define which columns need to be encoded
    cat_cols = make_column_selector(dtype_include='object')
    num_cols = make_column_selector(dtype_include='number')
    cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                            MinMaxScaler())
    cat_num = MinMaxScaler()

    # Create preprocessor pipeline
    preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

    classifier = LabelPowerset(RandomForestClassifier(max_features=1, min_samples_split=10, 
                                                      n_jobs=-1, random_state=42))

    pipeline = make_pipeline(preprocessing, classifier)
    return pipeline.fit(X_train, y_train)


In [22]:
file_name = 'XWines_Full_100K_wines.csv'
file_path = f'~/code/ArjanAngenent/VinoDine/raw_data/{file_name}'
wine_df, grape_column = create_binary_df_2(file_path)
wine_df

X_train, X_test, y_train, y_test = create_X_train_y_train_2(wine_df, grape_column)

model = train_model_2(X_train, y_train)

y_pred = model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.82      0.85      2602
           1       0.99      0.99      0.99     17390
           2       0.90      0.92      0.91      4952
           3       0.91      0.92      0.91     10912
           4       0.93      0.94      0.94     11641
           5       0.86      0.79      0.82      6743
           6       0.92      0.92      0.92      7656
           7       0.94      0.94      0.94     17504
           8       0.92      0.90      0.91      5575
           9       0.91      0.91      0.91      7469
          10       0.91      0.88      0.89      7469
          11       0.88      0.90      0.89      5491

   micro avg       0.93      0.92      0.92    105404
   macro avg       0.91      0.90      0.91    105404
weighted avg       0.93      0.92      0.92    105404
 samples avg       0.93      0.92      0.92    105404

Accuracy: 0.8167109556325474


In [23]:
def pred_2(model, X_pred: pd.DataFrame):

    y_pred = model.predict(X_pred)
    y_pred = y_pred.toarray()

    foods = ['Appetizer', 'Beef', 'Cured Meat', 'Game Meat', 'Lamb', 'Pasta', 'Pork', 'Poultry', 
             'Rich Fish', 'Shellfish', 'Veal', 'Vegetarian']
    foods_index = np.where(y_pred[0]==1)[0].tolist()
    foods_to_choose = []
    for i in foods_index:
        foods_to_choose.append(foods[i])

    return {"foods": foods_to_choose}

In [24]:
print(X_train.iloc[:,4:].columns.to_list())

['Abbuoto', 'Abouriou', 'Abrostine', 'Acolon', 'Agiorgitiko', 'Aglianico', 'Aidani', 'Airen', 'Albalonga', 'Albana', 'Albanella', 'Albariño', 'Albarola', 'Albarossa', 'Albarín Blanco', 'Albillo', 'Albillo Crimean', 'Albillo Mayor', 'Albillo de Albacete', 'Aleatico', 'Alfrocheiro Preto', 'Alibernet', 'Alicante Bouschet', 'Alicante Ganzin', 'Aligoté', 'Altesse', 'Alvarelhão', 'Alvarinho', 'Amigne', 'Ancellotta', 'Ansonica', 'Antão Vaz', 'Aragonez', 'Aramon', 'Arbane', 'Areni', 'Argaman', 'Arinarnoa', 'Arinto', 'Arinto de Bucelas', 'Arinto dos Açores', 'Arneis', 'Arnsburger', 'Arriloba', 'Aspiran Bouschet', 'Asprinio Bianco', 'Assario Branco', 'Assyrtiko', 'Athiri', 'Aurore', 'Avanà', 'Avesso', 'Avgoustiatis', 'Azal Branco', 'Azal Tinto', 'Babić', 'Bacchus', 'Baco Noir', 'Baga', 'Barbarossa', 'Barbera', 'Barcelo', 'Barsaglina', 'Bastardo Magarachsky', 'Batoca', 'Bellone', 'Bianca', 'Biancame', 'Bianchetta Trevigiana', "Bianco d'Alessano", 'Biancolella', 'Bical', 'Black Queen', 'Blauburger

In [12]:
def create_X_pred(Type: str, 
            ABV: float, 
            Body: str, 
            Acidity: str, 
            grapes: list):
    X_pred = pd.DataFrame.from_dict({'Type': [Type],
                                        'Body': [Body],
                                        'Acidity': [Acidity],
                                        'ABV': [ABV]},
                                    orient='columns')
    
    grapes_columns = ['Abbuoto', 'Abouriou', 'Abrostine', 'Acolon', 'Agiorgitiko', 'Aglianico', 'Aidani', 
              'Airen', 'Albalonga', 'Albana', 'Albariño', 'Aleatico', 'Alfrocheiro Preto', 'Alicante Bouschet', 
              'Aligoté', 'Alvarelhão', 'Alvarinho', 'Ancellotta', 'Ansonica', 'Antão Vaz', 'Aragonez', 'Arinto', 
              'Arinto de Bucelas', 'Arneis', 'Assyrtiko', 'Athiri', 'Avesso', 'Azal Branco', 'Bacchus', 
              'Baco Noir', 'Baga', 'Barbera', 'Bical', 'Blauburgunder', 'Blauer Portugieser', 'Blaufränkisch', 
              'Boal Branco', 'Bobal', 'Bombino Bianco', 'Bonarda', 'Bordô', 'Bourboulenc', 'Brachetto', 
              'Cabernet Franc', 'Cabernet Mitos', 'Cabernet Ruby', 'Cabernet Sauvignon', 'Caladoc', 'Callet', 
              'Camarate', 'Canaiolo Blanco', 'Canaiolo Nero', 'Cannonau', 'Carignan/Cariñena', 'Carmenère', 
              'Carricante', 'Castelão', 'Catarratto Bianco', 'Cayuga White', 'Cerceal Branco', 'Cesanese', 
              'Chambourcin', 'Chardonnay', 'Chasselas', 'Chenin Blanc', 'Ciliegiolo', 'Cinsault', 'Clairette', 
              'Colombard', 'Colorino del Valdarno', 'Concord', 'Cornalin', 'Cortese', 'Corvina', 'Corvinone', 
              'Couderc', 'Counoise', 'Croatina', 'Côdega de Larinho', 'Côt', 'Dindarella', 'Diolinoir', 
              'Dolcetto', 'Domina', 'Dona Blanca', 'Donzelinho Branco', 'Dornfelder', 'Drupeggio', 'Durella', 
              'Durif', 'Encruzado', 'Erbaluce', 'Espadeiro', 'Falanghina', 'Favorita', 'Fernão Pires', 
              'Feteasca Alba', 'Feteasca Neagra', 'Feteasca Regala', 'Fiano', 'Folle Blanche', 'Fragolino', 
              'Frappato', 'Freisa', 'Friulano/Sauvignonasse', 'Fumé Blanc', 'Furmint', 'Gaglioppo', 'Gaidouria', 
              'Gamaret', 'Gamay Noir', 'Garanoir', 'Garganega', 'Garnacha', 'Garnacha Blanca', 'Garnacha Tinta', 
              'Garnacha Tintorera', 'Gelber Muskateller', 'Gewürztraminer', 'Glera/Prosecco', 'Godello', 'Gouveio',
              'Graciano', 'Grauburgunder', 'Grecanico', 'Grechetto', 'Greco', 'Grenache', 'Grenache Blanc', 
              'Grenache Gris', 'Grignolino', 'Grillo', 'Grolleau', 'Groppello', 'Gros Manseng', 'Grüner Veltliner',
              'Hondarrabi Zuri', 'Hárslevelű', 'Incrocio Manzoni', 'Inzolia', 'Isabella', 'Jaen', 'Kadarka', 
              'Katsano', 'Kerner', 'Kisi', 'Koshu', 'Kékfrankos', 'Lacrima', 'Lagrein', 'Lambrusco', 'Lemberger', 
              'Limnio', 'Loureiro', 'Macabeo', 'Malagouzia', 'Malbec', 'Malvasia', 'Malvasia Fina', 
              'Malvasia Nera', 'Malvasia di Candia', 'Mammolo', 'Manto Negro', 'Marechal Foch', 'Maria Gomes', 
              'Marsanne', 'Marselan', 'Marzemino', 'Mataro', 'Mauzac Blanc', 'Mavrotragano', 'Mavrud', 'Mazuelo', 
              'Melnik', 'Melon de Bourgogne', 'Mencia', 'Merlot', 'Modrý Portugal', 'Molinara', 'Monastrell', 
              'Montepulciano', 'Moreto', 'Moschofilero', 'Mourisco', 'Mourvedre', 'Mtsvane Kakhuri', 'Muscadelle', 
              'Muscadine', 'Muscardin', 'Muscat Blanc', 'Muscat Noir', 'Muscat Orange', 'Muscat Ottonel', 
              'Muscat of Alexandria', 'Muscat of Frontignan', 'Muscat of Setúbal', 'Muscat/Moscatel Galego', 
              'Muscat/Moscatel Roxo', 'Muscat/Moscato', 'Muscat/Moscato Bianco', 'Muscat/Moscato Giallo', 
              'Muscat/Muscatel', 'Muscat/Muskat Moravsky', 'Müller-Thurgau', 'Nascetta', 'Nebbiolo', 
              'Negrara Trentino', 'Negrette', 'Negroamaro', 'Nerello Cappuccio', 'Nerello Mascalese', 
              "Nero d'Avola", 'Nero di Troia', 'Neuburger', 'Niagara', 'Nielluccio', 'Olasz Rizling', 
              'Ortega', 'Oseleta', 'Palomino', 'Parellada', 'Passerina', 'País/Mission', 'Pecorino', 
              'Pederna', 'Pedro Ximenez', 'Perrum', 'Petit Manseng', 'Petit Verdot', 'Petite Arvine', 
              'Petite Sirah', 'Picolit', 'Picpoul Blanc', 'Piedirosso', 'Pigato', 'Pignoletto', 'Pignolo', 
              "Pineau D'Aunis", 'Pinot Auxerrois', 'Pinot Blanc', 'Pinot Grigio', 'Pinot Gris', 'Pinot Meunier', 
              'Pinot Nero', 'Pinot Noir', 'Pinotage', 'Piquepoul Blanc', 'Plavac Mali', 'Posip Bijeli', 
              'Poulsard', 'Primitivo', 'Procanico', 'Prugnolo Gentile', 'Rabigato', 'Rabo de Ovelha', 
              'Raboso Piave', 'Raboso Veronese', 'Rebo', 'Refosco', 'Refosco dal Peduncolo Rosso', 'Regent', 
              'Ribolla Gialla', 'Riesling', 'Riesling Itálico', 'Rivaner', 'Rkatsiteli', 'Roditis', 'Rolle/Rollo', 
              'Rondinella', 'Rossignola', 'Rotgipfler', 'Roupeiro', 'Roussanne', 'Ruché', 'Rufete', 'Sagrantino', 
              'Samsó', 'Sangiovese', 'Saperavi', 'Sauvignon Blanc', 'Sauvignon Gris', 'Savagnin Blanc', 
              'Savatiano', 'Scheurebe', 'Schiava', 'Schioppettino', 'Schwarzriesling', 'Sciacarello', 'Sercial', 
              'Seyval Blanc', 'Silvaner/Sylvaner', 'Sousão', 'Souvignier Gris', 'Spätburgunder', 'St. Laurent', 
              'Susumaniello', 'Syrah/Shiraz', 'Sémillon', 'Síria', 'Tamaioasa Romaneasca', 'Tannat', 'Tempranillo',
              'Tempranillo Blanco', 'Teroldego', 'Terrantez', 'Terret', 'Tibouren', 'Tinta Amarela', 'Tinta Barroca', 
              'Tinta Caiada', 'Tinta Francisca', 'Tinta Madeira', 'Tinta Miúda', 'Tinta Negra Mole', 'Tinta Roriz', 
              'Tinta de Toro', 'Tinta del Pais', 'Tinto Cão', 'Tinto Fino', 'Tocai Friulano', 'Torrontés', 'Touriga Franca', 
              'Touriga Nacional', 'Trajadura', 'Traminer', 'Trebbiano', 'Trebbiano Toscano', "Trebbiano d'Abruzzo",
              'Treixadura', 'Trepat', 'Trincadeira', 'Trollinger', 'Trousseau', 'Ugni Blanc', 'Ull de Llebre', 
              'Valdiguie', 'Verdeca', 'Verdejo', 'Verdelho', 'Verdello', 'Verdicchio', 'Verduzzo Friulano', 
              'Vermentino', 'Vernaccia', 'Vernaccia di San Gimignano', 'Vernatsch', 'Vespolina', 'Vidal Blanc', 
              'Vignoles', 'Vinhão', 'Viognier', 'Viosinho', 'Vital', 'Vitovska', 'Viura', 'Vranac', 
              'Weissburgunder', 'Welschriesling', 'Xarel-lo', 'Xinomavro', 'Zibibbo', 'Zierfandler', 'Zinfandel', 
              'Zinfandel White', 'Zweigelt']
    
    X_pred = X_pred.join(pd.DataFrame(columns=grapes_columns))
    X_pred.fillna(0, inplace=True)
    X_pred[grapes] = 1
    
    return X_pred

In [13]:
X_pred = create_X_pred(Type='Red', Body='Full-bodied',Acidity='Low', ABV=14.5, grapes=['Cabernet Sauvignon' ] )

/var/folders/95/62wnky0d7yv7v6kjwmjy90m00000gn/T/ipykernel_2085/3750827143.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_pred.fillna(0, inplace=True)


In [14]:
X_test[X_test['Type']=='White']

,Type,ABV,Body,Acidity,Abbuoto,Abouriou,Abrostine,Acolon,Agiorgitiko,Aglianico,Aidani,Airen,Albalonga,Albana,Albanella,Albariño,Albarola,Albarossa,Albarín Blanco,Albillo,Albillo Crimean,Albillo Mayor,Albillo de Albacete,Aleatico,Alfrocheiro Preto,Alibernet,Alicante Bouschet,Alicante Ganzin,Aligoté,Altesse,Alvarelhão,Alvarinho,Amigne,Ancellotta,Ansonica,Antão Vaz,Aragonez,Aramon,Arbane,Areni,Argaman,Arinarnoa,Arinto,Arinto de Bucelas,Arinto dos Açores,Arneis,Arnsburger,Arriloba,Aspiran Bouschet,Asprinio Bianco,Assario Branco,Assyrtiko,Athiri,Aurore,Avanà,Avesso,Avgoustiatis,Azal Branco,Azal Tinto,Babić,Bacchus,Baco Noir,Baga,Barbarossa,Barbera,Barcelo,Barsaglina,Bastardo Magarachsky,Batoca,Bellone,Bianca,Biancame,Bianchetta Trevigiana,Bianco d'Alessano,Biancolella,Bical,Black Queen,Blauburger,Blauburgunder,Blauer Portugieser,Blaufränkisch,Boal Branco,Bobal,Bogazkere,Bombino Bianco,Bombino Nero,Bonamico,Bonarda,Bordô,Borraçal,Bosco,Bourboulenc,Bovale,Brachetto,Braquet,Braucol,Brianna,Bronner,Brun Argenté,Bruñal,Bual,Budai Zöld,Bukettraube,Burgund Mare,Busuioaca de Bohotin,Băbească Neagră,Cabernet Blanc,Cabernet Cortis,Cabernet Cubin,Cabernet Dorsa,Cabernet Franc,Cabernet Jura,Cabernet Mitos,Cabernet Ruby,Cabernet Sauvignon,Cabernet Severny,Cagnulari,Caiño Blanco,Caiño Tinto,Calabrese di Montenuovo,Caladoc,Calkarasi,Callet,Camarate,Canaiolo Blanco,Canaiolo Nero,Cannonau,Carignan/Cariñena,Carmenère,Carricante,Casavecchia,Cascade,Casetta,Castelão,Catarratto Bianco,Catawba,Cayuga White,Cencibel,Centesiminio,Cerceal Branco,Cesanese,Chambourcin,Chancellor,Charbono,Chardonel,Chardonnay,Chardonnay Musqué,Chasan,Chasselas,Chatus,Chenanson,Chenin Blanc,Chinuri,Cienna,Ciliegiolo,Cinsault,Clairette,Cococciola,Coda di Volpe Bianca,Colobel,Colombard,Coloraillo,Colorino del Valdarno,Concord,Corinto Nero,Cornalin,Cornifesto,Corot Noir,Cortese,Corvina,Corvinone,Couderc,Counoise,Criolla Grande,Croatina,Crouchen,Cynthiana,Côdega de Larinho,Côt,Dafni,Dakapo,De Chaunac,Debina,Diagalves,Dimiat,Dimrit,Dindarella,Diolinoir,Dolcetto,Domina,Dona Blanca,Donzelinho Branco,Donzelinho Tinto,Dornfelder,Drupeggio,Dunkelfelder,Duras,Durella,Durif,Dzvelshavi Obchuri,Edelweiss,Egiodola,Ehrenfelser,Emerald Riesling,Emir,Enantio,Encruzado,Erbaluce,Espadeiro,Falanghina,Falanghina Beneventana,Famoso,Favorita,Fenile,Fer Servadou,Fernão Pires,Feteasca Alba,Feteasca Neagra,Feteasca Regala,Fiano,Flora,Foglia Tonda,Fokiano,Folgasao,Folle Blanche,Fonte Cal,Fragolino,Francusa,Frappato,Fredonia,Freisa,Friulano/Sauvignonasse,Frontenac,Fruhroter Veltliner,Frühburgunder,Fumin,Fumé Blanc,Furmint,Gaglioppo,Gaidouria,Galotta,Gamaret,Gamay Noir,Gamay Teinturier de Bouze,Gamba di Pernice,Garanoir,Garganega,Garnacha,Garnacha Blanca,Garnacha Peluda,Garnacha Roja,Garnacha Tinta,Garnacha Tintorera,Garrido Fino,Gelber Muskateller,Gewürztraminer,Gigiac,Ginestra,Girgentina,Girò Blanc,Glera/Prosecco,Godello,Gold Traminer,Goldburger,Golubok,Gorgollasa,Goruli Mtsvane,Gouveio,Gouveio Real,Graciano,Grand Noir,Grasa de Cotnari,Grauburgunder,Grecanico,Grechetto,Grechetto Rosso,Greco,Greco Bianco,Greco Nero,Grenache,Grenache Blanc,Grenache Gris,Grignolino,Grillo,Gringet,Grolleau,Groppello,Gros Manseng,Gros Verdot,Grüner Veltliner,Guardavalle,Gutedel,Hanepoot,Helios,Hibernal,Hondarrabi Beltza,Hondarrabi Zuri,Humagne Blanche,Humagne Rouge,Huxelrebe,Hárslevelű,Incrocio Manzoni,Inzolia,Irsai Oliver,Isabella,Jacquère,Jaen,Jampal,Johannisberg,Johanniter,Juan Garcia,Kabar,Kadarka,Kakhet,Kakotrygis,Kalecik Karasi,Kangun,Karasakiz,Karmahyut,Katsano,Keratsuda,Kerner,Khikhvi,Királyleányka,Kisi,Klevner,Kokur Bely,Koshu,Kotsifali,Krasnostop Anapsky,Krasnostop Zolotovsky,Kratosija,Krstac,Kydonitsa,Kékfrankos,L'Acadie Blanc,Lacrima,Lafnetscha,Lagrein,Lambrusco,Lampia,Landot Noir,Lauzet,Leanyka,Lefkada,Lemberger,Len de l'El,Lenoir,Leon Millot,Liatiko,Limnio,Limniona,Listan Negro,Loin de l'Oeil,Lorena,Loureiro,Macabeo,Madeleine Angevine,Magliocco Canino,Malagouzia,Malbec,Malbo Gentile,Malvar,Malvasia,Malvasia Bian

In [15]:
pred_2(model, X_pred)

ValueError: columns are missing: {'Vidiano', 'Pampanuto', 'Palava', 'Orion', 'Triomphe', 'Dakapo', 'Sultana', 'Cabernet Jura', 'Bosco', 'Millot-Foch', 'Lenoir', 'Malvazija Istarska', 'Edelweiss', 'Petit Courbu', 'Bukettraube', 'Cynthiana', 'Caiño Tinto', 'Violeta', 'Kakhet', 'Smederevka', 'Debina', 'Ondenc', 'Petite Pearl', 'Narince', 'Pedral', 'Bellone', 'Borraçal', 'Parraleta', 'Casetta', 'Rotberger', 'Bianca', 'Rieslaner', 'Coloraillo', 'Thrapsathiri', 'Coda di Volpe Bianca', 'Ojaleshi', 'Albillo Crimean', 'Steuben', 'Trebbiano Giallo', "Roussette D'Ayze", 'Malbo Gentile', 'Keratsuda', 'Riesel', 'Brianna', 'Muscat Early', 'Michet', 'Sarba', 'Arinto dos Açores', 'Gutedel', 'Sumoll Blanc', 'Muscat/Moscatello Selvatico', 'Kokur Bely', 'Humagne Rouge', 'Ramisco', 'Muscat Black', "L'Acadie Blanc", 'Prieto Picudo', 'Robola', 'Mayolet', 'De Chaunac', 'Helios', 'Sacy', 'Tintilla', 'Donzelinho Tinto', 'Schönburger', 'Mavroudi Arachovis', 'Melody', 'Albarola', 'Peloursin', 'Pamid', 'Prié', 'M

# Without Grapes

In [ ]:
def create_binary_df(file_path):
    mlb_harm = MultiLabelBinarizer(sparse_output=False)
    mlb_grape = MultiLabelBinarizer(sparse_output=False)

    wine_df = pd.read_csv(file_path)

    wine_df.drop(columns=['Grapes'], inplace=True)

    # Binary encode Harmonize(kinds of food)
    wine_df_bin = wine_df.join(pd.DataFrame(
        mlb_harm.fit_transform(eval(element) for element in wine_df.Harmonize),
        index=wine_df.index,
        columns=mlb_harm.classes_
        ))
    wine_df_bin.drop(columns=['Harmonize'], inplace=True)
    
    # Create a list of the kind of grapes that are mentioned less then 2.000 times
    harm_list = wine_df_bin.iloc[:,15:].sum() # sum the number of times a food is mentioned via column
    harm_to_drop = harm_list[harm_list<=10_000].index.to_list() # create a list withe kind of food mentioned less then 50 times
    wine_df_bin.drop(columns=harm_to_drop, inplace=True) # drop columns with food not mentioned more then 50 times
    wine_df_bin = wine_df_bin[wine_df_bin.iloc[:,15:].eq(1).any(axis=1)] # drop wines which are not represented by a food anymore

    #Drop addional columns not used for model
    wine_df_bin_cleaned = wine_df_bin.drop(columns=['WineName', 'WineID','Code','Country','RegionID','RegionName','WineryID','Website','Vintages', 'WineryName'])

    return wine_df_bin_cleaned

In [ ]:
def create_X_train_y_train(df, test_size=0.3):
    X = df[['Type','Body','Acidity', 'ABV']]
    y = df.drop(columns=['Type','Elaborate','Body','Acidity', 'ABV'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=test_size)
    return X_train, X_test, y_train, y_test

In [ ]:
def train_model(X_train, y_train):
    # Create binary classifier
    # Define which columns need to be encoded
    cat_cols = make_column_selector(dtype_include='object')
    num_cols = make_column_selector(dtype_include='number')
    cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                            MinMaxScaler())
    cat_num = MinMaxScaler()

    # Create preprocessor pipeline
    preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

    classifier = LabelPowerset(SGDClassifier())
    #classifier = MLkNN(k=5)

    pipeline = make_pipeline(preprocessing, classifier)
    return pipeline.fit(X_train, y_train)

In [ ]:
def pred(model, X_pred: pd.DataFrame):

    y_pred = model.predict(X_pred)
    y_pred = y_pred.toarray()

    foods = ['Appetizer', 'Beef', 'Cured Meat', 'Game Meat', 'Lamb', 'Pasta', 'Pork', 'Poultry', 'Rich Fish', 'Shellfish', 'Soft Cheese', 'Spicy Food', 'Veal', 'Vegetarian']
    foods_index = np.where(y_pred[0]==1)[0].tolist()
    foods_to_choose = []
    for i in foods_index:
        foods_to_choose.append(foods[i])

    return {"foods": foods_to_choose}

In [ ]:
file_name = 'XWines_Full_100K_wines.csv'
file_path = f'~/code/ArjanAngenent/VinoDine/raw_data/{file_name}'
wine_df = create_binary_df(file_path)
wine_df


X_train, X_test, y_train, y_test = create_X_train_y_train(wine_df)

model = train_model(X_train, y_train)

y_pred = model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# X_pred
X_pred = pd.DataFrame.from_dict({'Type': ["Red"],
                                    'Body': ["Full-bodied"],
                                    'Acidity': ['Medium'],
                                    'ABV': [13.5]},
                                orient='columns')

y_pred = pred(model, X_pred)
# y_pred
print(y_pred)